#**CHAPTER 9. EVENT DRIVEN ARCHITECTURES**
---

##REFERENCE

https://chatgpt.com/share/699707f7-110c-8012-9616-979034c5a0fa

##0.CONTEXT

**Introduction — Treasury Liquidity Monitoring as an Event-Driven AI System (What This Notebook Demonstrates, in Plain Terms)**

If you have ever sat through a treasury meeting where everyone is looking at the same spreadsheet but arriving at different conclusions, you already know the core problem this notebook addresses: **liquidity is not a number, it is a moving target**. A cash balance snapshot can look fine at 9:00 a.m. and be wrong by 3:00 p.m. because cash moved, payables accelerated, receivables slipped, FX shifted, rates moved, or a lender requested a covenant test. Treasury reality is not “one calculation”; it is a **sequence of events** that progressively change the firm’s feasible actions. That is the practical reason event-driven monitoring exists in real finance teams: it ensures that as the world changes, the monitoring logic changes with it, and it does so in a disciplined, reviewable manner.

This notebook is a compact demonstration of how AI can be used to generate an event-driven analysis workflow that is **auditable, bounded, and deterministic**, while still benefiting from language models for communication tasks. In other words, it shows a professional division of labor: **the system computes and controls; the model explains and drafts**. The architecture is deliberately not “AI that guesses treasury answers.” It is **AI as a workflow engine** that enforces a treasury process you would recognize: ingest events, update the liquidity picture, test covenants, decide actions, notify stakeholders, and repeat until the event stream is exhausted or the run is bounded. Everything is explicit and inspectable.

To make this concrete for a committee, we can frame the notebook as a simulation of a real treasury analyst’s day, but implemented as a **state-driven workflow** rather than a human-driven checklist. The most important concept is this: **the notebook treats treasury monitoring as a state machine that responds to events**. Each new piece of information is an “event” that enters the system. The system updates its internal “state” (cash, revolver drawn, short-term investments, upcoming payables/receivables, covenant thresholds, and derived liquidity metrics). Based on that state, it routes to the correct next step. That routing is not left to the language model’s judgment or to vague prompt phrasing. It is performed via explicit conditional logic in the LangGraph topology. This is what makes the workflow explainable and defensible: **routing is decided by state and policies, not by conversational intuition**.

**What “event-driven” means in this notebook**

In treasury operations, “event-driven” means you do not wait for the end-of-day report to learn the firm is drifting into a liquidity warning. You react to material changes as they happen, and you do so consistently. In this notebook, an event is a structured record with a timestamp and a kind. Examples in the synthetic stream include:

- **Cash in** and **cash out** movements (operational inflows/outflows).
- **FX move** (a simplified representation of exposure translation impact).
- **Rate move** (a simplified representation of floating-rate sensitivity).
- **Covenant test request** (a forcing function that triggers visibility).

The point is not that these particular toy dynamics capture the entire reality of a multinational treasury function. The point is that the system is built in the same shape as a real monitoring system: **a queue of events** enters, and the workflow processes them sequentially with a bounded loop.

In the notebook’s state, there is an explicit **event index** and a **step counter**, and there is a **max_steps** bound. Those two controls matter because event-driven systems can be dangerous if they run forever or if they “thrash” in loops. This notebook is engineered to avoid that. It processes at most a defined number of events per run, and it always has an explicit END node. That design makes the workflow safe in a classroom and safe as a conceptual template for production teams.

**What AI is doing here — and what it is not doing**

A committee will reasonably ask: “Is this AI making financial decisions?” The answer is precise:

- The workflow’s **financial computations** are deterministic and coded: liquidity metrics, revolver utilization, covenant status classification, and action triggers are hard rules.
- The language model is used only for **notification drafting**, producing a terse “ops note” from a structured JSON snapshot.

That separation is intentional and is the strongest governance argument you can make in a professional context. If you want AI in treasury operations, you typically want it in places where humans spend time on repeated tasks: summarizing, drafting, formatting, turning structured data into consistent communications. You generally do not want a black box making capital-structure decisions. This notebook models that boundary explicitly.

So, the best way to describe the role of AI to your bosses is:

**The AI model is not the controller. The AI model is the communicator. The controller is the state machine.**

This is also why the notebook is built around a typed state schema. The system does not “remember” or “infer” what matters; it stores what matters as fields in a TypedDict. That makes the workflow auditable: you can print the state at any step, inspect what changed, and verify that the routing logic behaved correctly.

**Why a LangGraph topology matters for finance teams**

Many organizations have scripts, dashboards, and ad-hoc monitoring routines. What they often lack is a clear, explicit representation of the process: which steps happen in what order, what conditions cause branching, how retries are bounded, and what artifacts are produced. LangGraph provides a useful abstraction: a treasury monitoring workflow can be represented as a graph where each node is a small, testable function, and each edge is a state-driven transition. This matters in finance because it enables three operational outcomes:

1. **Reviewability.** A graph is easier to review than a long procedural script because the topology shows the logic at a glance.
2. **Governance.** You can attach controls to nodes and enforce a predictable sequence. You can show auditors and risk committees a diagram, not just code.
3. **Scalability.** As requirements grow (multiple currencies, multiple entities, multiple revolvers, multiple covenants), the graph can be extended without turning into a single monolithic function.

This notebook is the ninth in a series, and the architectural progression is deliberate. Earlier notebooks introduced loops, refusal boundaries, critique loops, tool wrappers, regime machines, committee aggregation, hub-and-spoke drafting, and retrieval routing. Notebook 9 adds a new dimension: **event-driven workflow**. That is a real operational shift. It is the difference between a model that answers a question when asked and a system that monitors continuously as new information arrives.

**Mapping the notebook to a real treasury monitoring process**

If we translate the notebook into human workflow language, it looks like this:

- **Step 1: Pull the next update** (a bank transaction, a cash forecast update, an FX move, a lender request).
- **Step 2: Apply it to the current treasury picture** (cash changes, forecast changes, exposures shift).
- **Step 3: Recompute liquidity** (what is available, what is committed, what is due).
- **Step 4: Test covenant policy** (are we OK, warning, or breach).
- **Step 5: Take policy actions if needed** (sell liquid investments, draw revolver within cap, slow payables in warning).
- **Step 6: Communicate status** (a short note for treasury leadership / CFO / risk).
- **Step 7: Move to the next update** until the event stream ends or the run bound is reached.

That is exactly what the graph implements. The value is not “AI answered a treasury question.” The value is “AI is embedded in a controlled workflow that continuously produces a consistent monitoring output.”

**The state schema is the backbone of governance**

The explicit TypedDict state is not just a coding preference; it is the governance backbone. In real finance organizations, the failures you worry about are:

- People applying different definitions of liquidity.
- People forgetting buffers or thresholds.
- People changing parameters without tracking.
- People escalating too late because the monitoring step was skipped.
- People missing a condition in a spreadsheet because it is hidden in a tab.

The notebook defends against these failure modes with an explicit state design:

- It stores **cash**, **short-term investments**, **revolver limit and drawn**, and **near-term AP/AR**.
- It stores policy thresholds like **minimum cash buffer**, **max revolver utilization**, and **minimum covenant liquidity**.
- It stores derived fields like **liquidity_usd**, **revolver_utilization**, and **covenant_status**.
- It stores **trace** records for each node execution, enabling after-action review.

This is the key point to communicate: in professional settings, the main risk is not “the model is wrong.” The main risk is “the process is unreviewable.” A typed, explicit state makes the process reviewable.

**What makes the workflow “deterministic” and why that matters**

Determinism in this notebook has two layers:

1. The event stream is generated with a fixed random seed. That means every run yields the same synthetic events unless the seed is changed intentionally.
2. The state transitions are pure functions of state and event. The same input state and event yield the same output state.

That matters because professional teams need to reproduce results. If the committee asks “why was this a breach,” you can point to the state and the event that caused it. If the committee asks “what happens if payables accelerate,” you can modify the event stream and rerun while maintaining audit artifacts. This is the opposite of ad-hoc analysis.

**The graph is the explanation**

One of the biggest advantages of this approach is that the workflow is not hidden. The diagram produced in the notebook is the learning artifact and the governance artifact. It shows:

- The system starts at **NEXT_EVENT**.
- If there is no event or the maximum steps are reached, it goes to **END**.
- If there is an event, it goes to **APPLY_EVENT** and then **METRICS**.
- It then goes to **COVENANT_GATE**.
- If status is OK, it goes to **NOTIFY** (the routine reporting path).
- If status is WARNING or BREACH, it goes to **ACTIONS**, then recomputes **METRICS**, then rechecks **COVENANT_GATE** (a bounded stabilization pass).
- Notifications route to **ADVANCE**, which increments the event index, and then loops back to **NEXT_EVENT** until stopping conditions are met.

This is the event-driven mechanism in one picture: it is a loop that is driven by the arrival of events, and it is bounded by explicit stop rules. That is the heart of the story to your bosses.

**Why there is a bounded stabilization pass after actions**

A treasury monitoring system should not perform endless self-correction. It should take a defined action and then verify whether the action moved the metrics into compliance. In the notebook, when the covenant gate reports WARNING or BREACH, the system takes deterministic actions (sell liquid investments, draw revolver within limits, or slow payables). After actions, it recomputes metrics and re-evaluates the covenant gate. That is a professional control: **act, then re-measure**. We do it once. We do not keep acting repeatedly in a loop because that would be a governance problem and could produce unrealistic behavior.

This is a key committee-facing point: the architecture is not “AI thrashing.” It is a controlled workflow that matches how a human team would operate under policy constraints.

**How to talk about “AI” without overselling**

Committees are rightly skeptical of AI hype. The framing that works is operational, not promotional:

- The system is a structured process engine.
- It uses a language model to reduce human workload on drafting and summarization.
- It produces artifacts that can be reviewed, logged, and audited.
- It avoids hidden memory and avoids implicit reasoning in routing.
- It makes escalation criteria explicit (OK/WARNING/BREACH), which can be aligned to policy and reviewed by risk.

In other words, the value proposition is: **more consistent monitoring, faster communication, better auditability**. That is a credible treasury benefit.

**What the artifact exports represent in an institutional workflow**

At the end of each run, the notebook exports three artifacts:

- **run_manifest.json** records what was run: configuration, versions, environment fingerprint, and bounds.
- **graph_spec.json** describes the topology (nodes and edges) so you can prove what workflow executed.
- **final_state.json** captures the final state: ending balances, covenant status, alerts, and trace.

These artifacts are not decorative. They are the bridge from a demo notebook to an institutional workflow. In production, you would store these in a run registry, attach them to a monitoring ticket, or include them in a weekly treasury pack. This is how you make “AI in finance” accountable: you treat runs as logged operations, not as one-off conversations.

**How a senior analyst should present this to leadership**

A simple narrative structure that maps directly to the notebook is:

1. **We are modeling treasury monitoring as a stream of events.**
2. **Each event updates a single state object that represents the treasury picture.**
3. **The workflow recomputes liquidity and tests covenants after every event.**
4. **If the state indicates risk, the workflow takes policy actions and re-measures once.**
5. **The AI model drafts a consistent operational note from the state snapshot.**
6. **We log everything: topology, run manifest, final state, and trace.**
7. **We stop deterministically: end of events or max steps. No runaway loops.**

This communicates competence and governance. It shows the committee that you understand the difference between using AI for **process augmentation** versus using AI as an unbounded decision-maker.

**Limitations (and why they are a feature, not a bug)**

This notebook is intentionally simplified. It uses one currency, toy dynamics for FX and rates, and a simple liquidity formula. In a committee setting, you should not apologize for that. You should emphasize that:

- The objective is to teach architecture: how to structure event-driven workflows.
- The placeholder economics can be replaced by your firm’s real treasury logic.
- The key is that the workflow structure remains correct: event ingestion, state update, gating, action, notification, logging.

The notebook also intentionally avoids a common failure mode: letting the language model decide what to do next. In many AI demos, routing is “prompt-based.” That is fragile. Here, routing is done by the graph’s conditional edges based on the state. That is exactly what you want in finance: **state drives control flow**.

**Why this matters now**

Treasury teams face increasing volatility, tighter lender scrutiny, and more complex liquidity structures. At the same time, expectations for speed and clarity of internal communication have increased. Event-driven monitoring is a natural response: it is the operational form of “always current.” AI can help, but only if it is embedded in a controlled process. This notebook is a demonstration of that safe integration pattern.

In a sentence you can use in a committee meeting:

**This notebook shows how to build an event-driven treasury monitoring system where the workflow is deterministic and auditable, the escalation logic is policy-based, and the language model is used only to generate consistent human-facing operational notes from structured state.**

That is a serious, committee-ready description. It does not oversell. It explains what is happening. It ties directly to the graph and the exported artifacts. And it makes clear that the architecture is the asset: the same pattern can be expanded to multi-entity cash pooling, multi-currency exposure ladders, covenant baskets, and automated evidence packs for lenders.

**What to watch for as you present the live run**

If you run the notebook in front of the committee, guide them to three “proof points”:

- The **diagram**: shows the workflow is not hand-waved. It is explicit.
- The **alerts**: show that after each event, the system classifies status and generates a note.
- The **artifacts**: show auditability and reproducibility, which are mandatory in finance.

If they ask “what is the AI doing,” open the **NOTIFY** node and show that it consumes a JSON snapshot and produces a short ops note. If they ask “what prevents this from going out of control,” point to **max_steps**, the **event_idx** termination, and the explicit **END** node. If they ask “how do we know what happened,” point to **trace** and **final_state.json**.

This is the professional story: not magic, not hype, but a disciplined event-driven system where AI is placed in the right role and the workflow is visible, bounded, and reviewable.


##1.LIBRARIES AND ENVIRONMENT

**CELL 1 — Colab-safe installs, conflict avoidance, and the “debug shim”**

Cell 1 is the notebook’s operational foundation. In a professional environment, the first failure mode is not “bad modeling,” it is **inconsistent runtime behavior**: Colab comes with preinstalled packages, and those versions can change over time. If we blindly force installs, we risk breaking the environment; if we do not manage versions at all, we risk subtle incompatibilities. This cell balances both by using a conservative policy: **install only what is missing or likely incompatible**, and otherwise leave Colab’s environment alone. That is why the cell checks versions first, then selectively installs only when major/minor differences suggest API breaks.

A key fix in this notebook is the small “LangChain debug shim.” The runtime error you encountered is a real-world example of why controlled environments matter: `langchain_core` expects `langchain.debug` to exist, but some Colab states can have a partial or mismatched `langchain` module. Instead of letting a small missing attribute crash the run, we add a minimal compatibility layer to ensure `langchain.debug` exists. This is not a hack for performance; it is a safety measure for **reproducibility** in teaching and committee demos. The shim is placed before importing LangGraph/LangChain components because import order matters.

This cell also establishes deterministic behavior. We set a fixed random seed and `PYTHONHASHSEED`, because in agentic systems, small nondeterministic differences can lead to different paths, different alerts, and different logs. In an audit mindset, you want to be able to say: “Given the same configuration, the same state transitions occur.” Determinism is therefore not cosmetic; it is part of governance.

Finally, Cell 1 prints the package versions and basic platform info. This is the start of your evidence trail: when you export `run_manifest.json` later, you want to be able to correlate the run with the environment. Committees care about this because it demonstrates professional discipline: the code is not merely “working,” it is **controlled and inspectable**.

In short, Cell 1 sets the conditions for everything else: stable dependencies, minimal conflict with Colab, deterministic behavior, and a guard against known runtime friction.


In [1]:
# CELL 1/10 — Colab-safe installs + imports (minimize conflicts) + LangChain debug shim

import sys, os, json, hashlib, uuid, platform, random, time, types as _types
import datetime as _dt
from typing import TypedDict, Literal, Dict, Any, List, Optional, Callable, Tuple

# -------------------------------------------------------------------
# Compatibility shim (must run BEFORE langgraph/langchain imports)
# Some Colab environments have a partial 'langchain' module missing `debug`,
# but langchain_core still tries to read `langchain.debug`.
# We ensure `langchain.debug` exists to avoid AttributeError.
# -------------------------------------------------------------------
try:
    import langchain as _langchain  # may exist but be incomplete
    if not hasattr(_langchain, "debug"):
        _langchain.debug = False
except Exception:
    _langchain = _types.ModuleType("langchain")
    _langchain.debug = False
    sys.modules["langchain"] = _langchain

# -------------------------------------------------------------------
# Install only what we need, and only when missing / major-minor mismatched.
# This avoids fighting Colab’s preinstalled stack.
# -------------------------------------------------------------------
import importlib.metadata as _md

def _ver(pkg: str) -> str:
    try:
        return _md.version(pkg)
    except Exception:
        return "missing"

_REQUIRED = {
    "langgraph": "0.2.39",
    "langchain": "0.3.14",        # include full langchain to reduce globals friction
    "langchain-core": "0.3.40",
    "anthropic": "0.34.2",
    "httpx": "0.27.2",
    "httpcore": "1.0.5",
}

def _pip_install(spec: str) -> None:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", spec])

def _mm(v: str) -> str:
    # major.minor helper (handles "missing")
    if v == "missing":
        return v
    parts = v.split(".")
    return ".".join(parts[:2]) if len(parts) >= 2 else v

to_install: List[str] = []
for pkg, want in _REQUIRED.items():
    have = _ver(pkg)
    if have == "missing":
        to_install.append(f"{pkg}=={want}")
    else:
        # Only pin when major/minor mismatch suggests API breakage.
        if _mm(have) != _mm(want):
            to_install.append(f"{pkg}=={want}")

if to_install:
    for spec in to_install:
        _pip_install(spec)

# -------------------------------------------------------------------
# Imports (after shims + installs)
# -------------------------------------------------------------------
from langgraph.graph import StateGraph, END
from google.colab import userdata
from IPython.display import HTML, display

# Determinism
random.seed(9)
os.environ["PYTHONHASHSEED"] = "9"

print("VERSIONS:", {k: _ver(k) for k in _REQUIRED})
print("LANGCHAIN_DEBUG_SHIM:", getattr(__import__("langchain"), "debug", None))
print("PYTHON:", sys.version.split()[0], "| PLATFORM:", platform.platform())


VERSIONS: {'langgraph': '0.2.39', 'langchain': '0.3.14', 'langchain-core': '0.3.63', 'anthropic': '0.34.2', 'httpx': '0.27.2', 'httpcore': '1.0.9'}
LANGCHAIN_DEBUG_SHIM: False
PYTHON: 3.12.12 | PLATFORM: Linux-6.6.105+-x86_64-with-glibc2.35


##2.CONFIGURATION

###2.1.OVERVIEW

**CELL 2 — Configuration, run identity, and the explicit TypedDict state schema**

Cell 2 defines the notebook’s governance contract. It introduces the model lock (`claude-haiku-4-5-20251001`) and the secret retrieval (`userdata.get("ANTHROPIC_API_KEY")` in all caps), but the deeper point is that the notebook does not treat “AI” as magic. It treats AI as one component in a controlled system. The configuration dictionary (`CONFIG`) collects the parameters that determine behavior: model name, seed, loop bounds, Mermaid version, and policy thresholds. We also compute a configuration hash. That hash is important because it creates a stable fingerprint: two runs with the same hash should behave the same. This is foundational for auditability.

The centerpiece of Cell 2 is the **TypedDict state schema**. In agentic architectures, state is the truth. Instead of passing loose dictionaries or conversational text, we define exactly what the system knows and what it can change. TreasuryState includes: event queue and indices, step counters and max bounds, balance sheet proxies (cash, revolver drawn/limit, short-term investments), short-term working capital expectations (AP/AR due in 7 days), policy thresholds (cash buffer, utilization cap, minimum liquidity covenant), and derived metrics (liquidity, utilization, covenant status). It also includes governance fields: last event, last action, alerts, and a trace.

This explicit schema is what makes the system “state-driven” rather than “prompt-driven.” In many AI demos, the model implicitly decides what matters. Here, the system decides what matters by schema design. If a variable is not in the schema, it cannot silently influence routing. That is a powerful professional property: it prevents hidden logic.

The initialization at the end of the cell constructs `INIT_STATE` with concrete numbers and deterministic defaults. This is your starting treasury snapshot and policy context. Everything the system does later is a transformation of this state. Committees care about this because it mirrors how treasury teams operate: you start from a known position, then events change it.

Finally, the cell prints run identifiers and confirms whether the API key is present. That is practical for live demonstrations: if the key is absent, the notebook still runs, and the model calls degrade gracefully to “LLM disabled.” This ensures your demo does not collapse due to secrets management.


###2.2.CODE AND IMPLEMENTATION

In [2]:
# CELL 2/10 — Config + explicit TypedDict state (deterministic, auditable)

MODEL_NAME = "claude-haiku-4-5-20251001"  # strict lock
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")  # ALL CAPS (may be None in classroom)

def utc_now_iso() -> str:
    return _dt.datetime.now(_dt.timezone.utc).isoformat()

def stable_hash(obj: Any) -> str:
    payload = json.dumps(obj, sort_keys=True, separators=(",", ":")).encode("utf-8")
    return hashlib.sha256(payload).hexdigest()

RUN_ID = f"run_{uuid.uuid4().hex[:12]}"
SEED = 9

class TreasuryEvent(TypedDict):
    ts_utc: str
    kind: Literal["CASH_IN", "CASH_OUT", "FX_MOVE", "RATE_MOVE", "COVENANT_TEST"]
    amount: float
    currency: Literal["USD"]
    meta: Dict[str, Any]

class TreasuryState(TypedDict):
    run_id: str
    ts_utc: str

    # queue + loop control
    events: List[TreasuryEvent]
    event_idx: int
    steps: int
    max_steps: int

    # balance sheet / liquidity snapshot (toy but structured)
    cash_usd: float
    revolver_limit_usd: float
    revolver_drawn_usd: float
    st_investments_usd: float   # highly liquid securities
    ap_due_7d_usd: float        # payables due next 7 days
    ar_due_7d_usd: float        # receivables due next 7 days

    # covenant + policy thresholds
    min_cash_buffer_usd: float
    max_revolver_utilization: float  # e.g., 0.80
    covenant_min_liquidity_usd: float

    # derived metrics
    liquidity_usd: float
    revolver_utilization: float
    covenant_status: Literal["OK", "WARNING", "BREACH"]

    # governance / trace
    last_event: Optional[TreasuryEvent]
    last_action: Optional[str]
    alerts: List[Dict[str, Any]]
    trace: List[Dict[str, Any]]

CONFIG = {
    "model": MODEL_NAME,
    "seed": SEED,
    "max_steps": 30,          # bounded loop: process at most N events per run
    "mermaid_version": "10.6.1",
    "policy": {
        "min_cash_buffer_usd": 5_000_000.0,
        "max_revolver_utilization": 0.80,
        "covenant_min_liquidity_usd": 10_000_000.0,
    },
}
CONFIG_HASH = stable_hash(CONFIG)

print("RUN_ID:", RUN_ID)
print("CONFIG_HASH:", CONFIG_HASH)
print("API_KEY_PRESENT:", bool(ANTHROPIC_API_KEY))


RUN_ID: run_85d4c9a4a97f
CONFIG_HASH: 4c0263de6650ed434f734cc7cc77fbc16196b1871feedbf691427424db4494cc
API_KEY_PRESENT: True


##3.VISUALIZATION STANDARDS

###3.1.OVERVIEW

**CELL 3 — Visualization standard: robust Mermaid rendering and graph-to-diagram extraction**

Cell 3 is not a cosmetic add-on. In this project, visualization is a learning artifact and a governance artifact. The entire series is about modular, scalable workflows, and the fastest way to understand a workflow is to see its topology. That is why we harden Mermaid rendering and require `display_langgraph_mermaid(graph)` behavior: the committee should be able to verify that the diagram matches the system’s actual control logic.

This cell provides two critical functions. First is `display_langgraph_mermaid`, which takes Mermaid code and renders it in Colab using the Mermaid ESM module pinned to a specific version. Pinning matters: Mermaid output can change across versions, and we want stable display behavior. The renderer is hardened in two ways: it uses `mermaid.render()` directly (instead of relying on `startOnLoad`), and it includes an error fallback that prints the raw Mermaid text if rendering fails. In professional terms, that means **no silent failures**: if visualization breaks, you still have the diagram definition as evidence.

Second is `_graph_to_mermaid`, which extracts Mermaid syntax from the compiled LangGraph object. LangGraph versions differ slightly in how they expose Mermaid output, so this function attempts common access patterns. If extraction fails, we return a minimal placeholder so the notebook remains runnable. This is another governance principle: do not let a non-core feature (diagram rendering) prevent the operational workflow from running. You can always inspect `graph_spec.json` later, but you still want the run to complete.

We also adapt the display style to your presentation requirement (white background, black text). This matters in committee settings where screenshots go into decks or PDFs. A readable diagram is a practical necessity.

In the context of the notebook, Cell 3 prepares the environment so that when we compile the graph in Cell 8, we can immediately show the graph and confirm topology. The committee can literally trace: “This is where events enter; this is where covenants are tested; this is where actions occur; this is where we stop.” That transparency is what differentiates an engineered workflow from a conversational AI demo.

In short, Cell 3 provides a reliable “glass box” view of the workflow and prevents the common failure mode of AI systems: hidden control logic that no one can audit or explain.


###3.2.CODE AND IMPLEMENTATION

In [17]:
# FIX — REWRITE CELL 3 (includes BOTH display_langgraph_mermaid + _graph_to_mermaid)
# Then re-run CELL 8.  (Your CELL 8 expects _graph_to_mermaid to exist.)

from IPython.display import HTML, display
import uuid as _uuid

def display_langgraph_mermaid(mermaid_code: str, *, mermaid_version: str = "10.6.1", theme: str = "default") -> None:
    """
    Hardened Colab Mermaid ESM renderer.
    - Renders via mermaid.render() (not startOnLoad)
    - theme="default" => white background + black text
    - theme="dark"    => dark background
    """
    code = mermaid_code.replace("</script>", "<\\/script>")

    block_id = f"mmd_{_uuid.uuid4().hex[:10]}"
    pre_id = f"{block_id}_pre"
    out_id = f"{block_id}_out"

    bg = "#ffffff" if theme == "default" else "#0b0d16"
    fg = "#111111" if theme == "default" else "#eef0ff"
    border = "rgba(0,0,0,0.12)" if theme == "default" else "rgba(255,255,255,0.10)"
    err = "#b00020" if theme == "default" else "#ffb4b4"

    html = f"""
<div style="background:{bg};border:1px solid {border};border-radius:14px;padding:14px;overflow-x:auto;">
  <pre id="{pre_id}" style="display:none; white-space:pre-wrap; color:{fg};">{code}</pre>
  <div id="{out_id}" style="color:{fg};"></div>
</div>

<script type="module">
  import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@{mermaid_version}/dist/mermaid.esm.min.mjs";

  const pre = document.getElementById("{pre_id}");
  const out = document.getElementById("{out_id}");
  const src = pre ? pre.textContent : "";

  mermaid.initialize({{
    startOnLoad: false,
    securityLevel: "strict",
    theme: "{theme}",
    flowchart: {{ curve: "basis" }},
    fontFamily: "ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial"
  }});

  try {{
    const res = await mermaid.render("{block_id}_svg", src);
    out.innerHTML = res.svg;
  }} catch (e) {{
    const safe = src.replaceAll("<","&lt;").replaceAll(">","&gt;");
    out.innerHTML =
      "<div style='color:{err}; font-weight:700; margin-bottom:8px;'>Mermaid render failed</div>" +
      "<pre style='white-space:pre-wrap; color:{fg}; opacity:0.92;'>" + safe + "</pre>";
    console.error("Mermaid error:", e);
  }}
</script>
"""
    display(HTML(html))

def _graph_to_mermaid(compiled_graph) -> str:
    """
    Robust extraction across LangGraph versions.
    Returns Mermaid 'flowchart' text.
    """
    # Common case: compiled_graph.get_graph().draw_mermaid()
    try:
        g = compiled_graph.get_graph()
        for fn_name in ("draw_mermaid", "draw_mermaid_png", "draw_mermaid_svg"):
            if hasattr(g, fn_name) and fn_name == "draw_mermaid":
                try:
                    return getattr(g, fn_name)()
                except Exception:
                    pass
        # Some versions keep mermaid text under internal fields
        for attr in ("mermaid", "_mermaid", "mermaid_str"):
            if hasattr(g, attr):
                v = getattr(g, attr)
                if isinstance(v, str) and v.strip():
                    return v
    except Exception:
        pass

    # Fallback: minimal placeholder (keeps notebook runnable)
    return "flowchart TD\n  A[Graph] --> B[Mermaid export unavailable]\n"

print("CELL 3 loaded: Mermaid renderer + _graph_to_mermaid ready (theme=default for white background).")


CELL 3 loaded: Mermaid renderer + _graph_to_mermaid ready (theme=default for white background).


##4.SYNTHETIC TREASURY EVENT

###4.1.OVERVIEW

**CELL 4 — Synthetic treasury event stream: deterministic operational “reality”**

Cell 4 creates the event stream that drives the entire notebook. The purpose is pedagogical and architectural: treasury monitoring is not a single computation; it is a **sequence** of updates. This cell generates that sequence deterministically so that runs are reproducible and classroom-friendly. The event stream is intentionally synthetic because we want the architecture to be shareable without confidential data. But it is still structured like real treasury signals: cash inflows/outflows, FX moves, rate moves, and covenant test triggers.

The function `make_events(seed, n)` uses a fixed seed and a fixed base timestamp, then creates events every few hours. Each event includes a timestamp, a kind, an amount (for cash movements), and a metadata dictionary for rates and FX. This resembles a simplified event bus: in real life, events might come from bank APIs, ERP postings, market data feeds, or lender communications. The important point is that **events are structured**. We do not feed the system raw text like “cash moved.” We feed typed events that can be applied deterministically to state. That is how you keep the system auditable.

The distribution of event kinds is chosen to resemble operational reality: more cash movements than FX/rate shocks, with periodic covenant tests. Covenant tests are important as a forcing function: even if nothing dramatic happens, treasury teams often run covenant checks because of calendar cycles or lender requests. By including “COVENANT_TEST” as an event type, we show that the workflow can be triggered both by market moves and by process requirements.

After generating `EVENTS`, the cell initializes the starting balance sheet proxy and the working capital proxy in `INIT_STATE`. These are not perfect representations of all treasury complexity, but they are sufficient to demonstrate the workflow: liquidity changes as cash changes, as AP/AR expectations shift, and as policy thresholds are applied. The state contains both the raw components and derived fields (liquidity and utilization are computed later). That separation is important: it prevents stale computed values from being treated as truth.

For a committee, the key message is: Cell 4 creates a repeatable “day in the life” stream. When you run the notebook, you are not asking a model a question; you are running a controlled monitoring cycle across an event queue. That is exactly what event-driven treasury systems do operationally.


###4.2.CODE AND IMPLEMENTATION

In [12]:
# CELL 4/10 — Deterministic synthetic treasury event stream (event-driven input)

def make_events(seed: int, n: int) -> List[TreasuryEvent]:
    rng = random.Random(seed)
    base = _dt.datetime(2026, 2, 19, 14, 0, tzinfo=_dt.timezone.utc)

    events: List[TreasuryEvent] = []
    for i in range(n):
        ts = (base + _dt.timedelta(hours=i * 3)).isoformat()
        kind = rng.choices(
            ["CASH_IN", "CASH_OUT", "FX_MOVE", "RATE_MOVE", "COVENANT_TEST"],
            weights=[28, 34, 10, 10, 18],
            k=1
        )[0]
        if kind == "CASH_IN":
            amt = float(rng.randint(250_000, 3_000_000))
        elif kind == "CASH_OUT":
            amt = -float(rng.randint(300_000, 3_500_000))
        else:
            amt = 0.0

        meta: Dict[str, Any] = {}
        if kind == "FX_MOVE":
            meta["usd_strength_pct"] = round(rng.uniform(-1.2, 1.2), 3)
        if kind == "RATE_MOVE":
            meta["sofr_shift_bps"] = int(rng.randint(-15, 25))
        if kind == "COVENANT_TEST":
            meta["test_reason"] = rng.choice(["Month-end", "Board pack", "Lender request", "Stress drill"])

        events.append({
            "ts_utc": ts,
            "kind": kind, "amount": amt, "currency": "USD",
            "meta": meta
        })
    return events

EVENTS = make_events(SEED, n=18)

INIT_STATE: TreasuryState = {
    "run_id": RUN_ID,
    "ts_utc": utc_now_iso(),

    "events": EVENTS,
    "event_idx": 0,
    "steps": 0,
    "max_steps": int(CONFIG["max_steps"]),

    "cash_usd": 12_000_000.0,
    "revolver_limit_usd": 25_000_000.0,
    "revolver_drawn_usd": 5_000_000.0,
    "st_investments_usd": 8_000_000.0,
    "ap_due_7d_usd": 6_500_000.0,
    "ar_due_7d_usd": 4_200_000.0,

    "min_cash_buffer_usd": float(CONFIG["policy"]["min_cash_buffer_usd"]),
    "max_revolver_utilization": float(CONFIG["policy"]["max_revolver_utilization"]),
    "covenant_min_liquidity_usd": float(CONFIG["policy"]["covenant_min_liquidity_usd"]),

    "liquidity_usd": 0.0,
    "revolver_utilization": 0.0,
    "covenant_status": "OK",

    "last_event": None,
    "last_action": None,
    "alerts": [],
    "trace": [],
}

print("EVENTS:", len(EVENTS))
print("INIT cash_usd:", INIT_STATE["cash_usd"], "revolver_drawn_usd:", INIT_STATE["revolver_drawn_usd"])


EVENTS: 18
INIT cash_usd: 12000000.0 revolver_drawn_usd: 5000000.0


##5.AGENT NODES

###5.1.OVERVIEW

**CELL 5 — Anthropic call wrapper and the AgentNode abstraction (auditable nodes)**

Cell 5 introduces two components that define how “AI” fits into a governed workflow. First, we implement a minimal HTTP wrapper for the Anthropic Messages API. Second, we implement the **AgentNode abstraction** that standardizes how every node in the graph executes and logs.

The Anthropic wrapper is intentionally small and guarded. It accepts a system prompt, a user message, and configuration knobs like max tokens and temperature. Crucially, it has a deterministic fallback: if the API key is missing, it returns a predictable “LLM disabled” response rather than crashing. This is essential for live demos and classroom settings: secrets management should not make the architecture unreadable. The wrapper also returns structured fields (`ok`, `model`, `content_text`, `usage`) so that downstream nodes can treat the LLM as a controlled tool, not a magical oracle.

The second part—AgentNode—is the deeper architectural point. In LangGraph, nodes are functions. But in professional systems, you want every node to be **traceable**. AgentNode wraps any node function and ensures we record a small trace entry before returning. The trace includes node name, timestamp, event index, step count, key financial values, and the last action. This makes debugging and governance much easier: you can inspect the trace to see which nodes ran, in what order, and what the state looked like when they ran.

This is a meaningful improvement over “print statements” or ad-hoc logs. A trace stored in state can be exported in `final_state.json` and audited later. It also helps explain behavior to a committee: you can show that the system never “jumped” steps or invented transitions; it followed the graph.

Cell 5 therefore formalizes the principle that runs must be reviewable. It also demonstrates an important boundary: the LLM is just one tool. The system’s logic is not contained in prompts; it is contained in state transitions and node sequencing. This is how you keep AI safe in finance contexts: you treat model calls as a controlled subroutine, and you wrap every execution with trace and structure.

In short, Cell 5 makes the workflow “institutional”: it is not just a set of functions, it is a set of standardized, trace-producing nodes that can be run and reviewed like a professional process.


###5.2.CODE AND IMPLEMENTATION

In [13]:
# CELL 5/10 — Anthropic Messages call (guarded) + AgentNode abstraction (LangGraph-friendly)

import httpx

def anthropic_messages(
    api_key: Optional[str],
    model: str,
    system: str,
    user: str,
    *,
    max_tokens: int = 220,
    temperature: float = 0.2,
    timeout_s: float = 20.0
) -> Dict[str, Any]:
    # Deterministic fallback when key missing (classroom-safe).
    if not api_key:
        return {
            "ok": True,
            "model": model,
            "content_text": "[LLM disabled: missing ANTHROPIC_API_KEY] " + user[:220],
            "usage": {"input_tokens": 0, "output_tokens": 0},
        }

    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json",
    }
    payload = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "system": system,
        "messages": [{"role": "user", "content": user}],
    }

    with httpx.Client(timeout=timeout_s) as client:
        r = client.post(url, headers=headers, json=payload)
        if r.status_code >= 300:
            return {"ok": False, "status": r.status_code, "text": r.text, "model": model}

        data = r.json()
        parts = data.get("content", [])
        text = ""
        for p in parts:
            if isinstance(p, dict) and p.get("type") == "text":
                text += p.get("text", "")
        return {
            "ok": True,
            "model": data.get("model", model),
            "content_text": text.strip(),
            "usage": data.get("usage", {}),
        }

class AgentNode:
    def __init__(self, name: str, fn: Callable[[TreasuryState], TreasuryState]):
        self.name = name
        self.fn = fn

    def __call__(self, state: TreasuryState) -> TreasuryState:
        # Small, auditable trace record per node
        before = {
            "node": self.name,
            "ts_utc": utc_now_iso(),
            "event_idx": state["event_idx"],
            "steps": state["steps"],
            "cash_usd": round(state["cash_usd"], 2),
            "liquidity_usd": round(state["liquidity_usd"], 2),
            "covenant_status": state["covenant_status"],
            "last_action": state["last_action"],
        }
        nxt = self.fn(state)
        nxt["trace"].append(before)
        return nxt

print("AgentNode abstraction ready.")


AgentNode abstraction ready.


##6.EVENT DRIVEN AGENT

###6.1.OVERVIEW

**CELL 6 — Core event-driven agents: NEXT_EVENT, APPLY_EVENT, METRICS**

Cell 6 is where the event-driven mechanism becomes concrete. We define three core nodes that mirror how a human treasury analyst processes updates: pull the next update, apply it, then recompute the metrics that matter. The cell is intentionally simple because it is the “engine room” of the monitoring loop.

`NEXT_EVENT` increments the step counter and sets `last_event` by reading the next event in the queue. This node embodies the idea that the workflow is not driven by a user’s question. It is driven by the arrival of new information. The event index is part of state, so the system always knows where it is in the stream. If there is no event, `last_event` becomes None, which later triggers termination in the graph.

`APPLY_EVENT` takes the `last_event` and applies it to state. In real systems, this is where operational reality enters your treasury snapshot. Here, we implement simplified transformations:
- Cash in/out directly changes `cash_usd`.
- FX moves adjust receivables slightly to mimic translation exposure.
- Rate moves increase payables pressure slightly to mimic floating-rate supplier effects.
- Covenant test events do not change values but change workflow behavior by forcing evaluation and reporting.

The important design principle is that the transformations are deterministic and explicit. That makes the workflow explainable. A committee can ask, “what changed cash?” and you can point to the cash event and the apply function. There is no hidden LLM reasoning in these updates.

`METRICS` recomputes the derived fields. This is a critical practice: derived metrics should be computed from underlying components, not stored as primary truth. The liquidity calculation is a toy definition, but it is explicit: cash plus short-term investments plus net near-term working capital minus a required cash buffer. Revolver utilization is computed from drawn and limit.

This structure teaches a professional pattern: you separate state updates from derived metric computation. That allows you to change definitions centrally and ensures consistency. It also mirrors how treasury dashboards work: they ingest raw balances and forecasts, then compute liquidity and policy ratios.

Finally, each node is wrapped as an AgentNode, so every pass through these steps produces trace entries. That is important because these nodes run repeatedly in the loop; you want a consistent record of how the state evolved event by event.

In short, Cell 6 defines the operational cadence: event arrival → state update → metric recomputation. Everything else in the notebook builds on this cadence.


###6.2.CODE AND IMPLEMENTATION

In [14]:
# CELL 6/10 — Core event-driven agents: NEXT_EVENT → APPLY_EVENT → METRICS

def _peek_event(state: TreasuryState) -> Optional[TreasuryEvent]:
    i = state["event_idx"]
    if i < 0 or i >= len(state["events"]):
        return None
    return state["events"][i]

def node_next_event(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    state["ts_utc"] = utc_now_iso()
    state["steps"] += 1
    ev = _peek_event(state)
    state["last_event"] = ev
    return state

def node_apply_event(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    ev = state["last_event"]
    if ev is None:
        state["last_action"] = "NO_EVENT"
        return state

    kind = ev["kind"]
    amt = float(ev["amount"])
    if kind in ("CASH_IN", "CASH_OUT"):
        state["cash_usd"] = float(state["cash_usd"] + amt)
        state["last_action"] = f"APPLIED_{kind}:{amt:+.0f}"
    elif kind == "FX_MOVE":
        # Toy: FX move affects near-term AR value slightly (translation exposure)
        shock = float(ev["meta"].get("usd_strength_pct", 0.0))
        state["ar_due_7d_usd"] = float(state["ar_due_7d_usd"] * (1.0 - 0.002 * shock))
        state["last_action"] = f"FX_MOVE usd_strength_pct={shock}"
    elif kind == "RATE_MOVE":
        # Toy: rate shift affects AP timing pressure (floating-rate suppliers)
        bps = int(ev["meta"].get("sofr_shift_bps", 0))
        state["ap_due_7d_usd"] = float(state["ap_due_7d_usd"] * (1.0 + 0.0006 * max(bps, 0)))
        state["last_action"] = f"RATE_MOVE sofr_shift_bps={bps}"
    elif kind == "COVENANT_TEST":
        state["last_action"] = f"COVENANT_TEST:{ev['meta'].get('test_reason','')}"
    else:
        state["last_action"] = f"UNKNOWN_EVENT:{kind}"
    return state

def node_compute_metrics(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    # Liquidity definition (toy but explicit): cash + ST investments + (AR - AP) - minimum cash buffer
    liquidity = state["cash_usd"] + state["st_investments_usd"] + (state["ar_due_7d_usd"] - state["ap_due_7d_usd"]) - state["min_cash_buffer_usd"]
    state["liquidity_usd"] = float(liquidity)

    util = 0.0
    if state["revolver_limit_usd"] > 0:
        util = float(state["revolver_drawn_usd"] / state["revolver_limit_usd"])
    state["revolver_utilization"] = util

    return state

NEXT_EVENT = AgentNode("NEXT_EVENT", node_next_event)
APPLY_EVENT = AgentNode("APPLY_EVENT", node_apply_event)
METRICS = AgentNode("METRICS", node_compute_metrics)

print("Core agents wired.")


Core agents wired.


##7.COVENANT GATE LIQUIDITY ACTIONS AND NOTIFICATIONS

###7.1.OVERVIEW

**CELL 7 — Covenant gate, deterministic actions, and LLM-backed notifications**

Cell 7 introduces the “decision and communication” layer that sits on top of core monitoring. In treasury terms, it answers two questions after every event: **Are we within policy? If not, what do we do? And who do we tell?** This cell does that in a governed way by splitting the logic into three nodes: a covenant gate, an actions node, and a notify node.

`COVENANT_GATE` converts numeric state into a discrete operational status: OK, WARNING, or BREACH. This is not an LLM judgment. It is a deterministic classification based on thresholds stored in state: minimum covenant liquidity and revolver utilization comfort. The use of discrete statuses is important because it reduces ambiguity. Finance teams need consistent escalation categories; they cannot rely on ad-hoc language.

`ACTIONS` implements a deterministic policy playbook. In this notebook, the playbook is intentionally simple but structurally realistic: in breach, prioritize selling liquid investments to raise cash, then draw revolver within utilization caps; in warning, either draw a small buffer or slow payables if utilization is already high. Two governance points matter here:
- Actions are bounded by explicit caps (max revolver utilization).
- Actions are policy-based, not model-based, so they can be reviewed and approved by leadership.

We then recompute metrics after actions and re-check covenant status (wired in Cell 8). This matches a real control practice: **act, then measure**.

`NOTIFY` demonstrates the appropriate role of the LLM in an institutional setting. The node packages a structured JSON snapshot (cash, liquidity, utilization, status, action, event details) and asks the model for a terse operations note. This is exactly where language models excel: turning structured data into consistent human-readable updates. The model is not asked to compute liquidity or decide actions; it is asked to draft communication. That separation is critical for committee acceptance.

We also store the alert in state with severity mapped from covenant status and include the LLM output. This ensures that the model’s contribution is logged and reviewable. If the model produces an odd note, you can inspect the exact input snapshot used and the output produced. That is how you keep LLM usage accountable.

Cell 7 therefore extends the workflow from “monitoring” to “operational response and reporting.” It also sets the stage for future autonomy: once you have explicit statuses and explicit actions, you can later introduce more advanced decision proposals or routing while maintaining the same control boundaries.


###7.2.CODE AND IMPLEMENTATION

In [15]:
# CELL 7/10 — Covenant gate + liquidity actions + LLM-backed notification (guarded)

def node_covenant_gate(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    liq = float(state["liquidity_usd"])
    util = float(state["revolver_utilization"])

    if liq < state["covenant_min_liquidity_usd"]:
        state["covenant_status"] = "BREACH"
    elif liq < (state["covenant_min_liquidity_usd"] + 2_000_000.0) or util > (state["max_revolver_utilization"] - 0.05):
        state["covenant_status"] = "WARNING"
    else:
        state["covenant_status"] = "OK"
    return state

def node_liquidity_actions(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    liq = float(state["liquidity_usd"])
    util = float(state["revolver_utilization"])

    action = "NO_ACTION"

    # Deterministic action policy (governed, inspectable):
    # 1) If breach: sell ST investments first, then draw revolver within utilization cap.
    # 2) If warning: pre-emptively draw small buffer OR slow AP payments (modeled as reducing AP due).
    if state["covenant_status"] == "BREACH":
        if state["st_investments_usd"] > 0:
            sell = min(state["st_investments_usd"], max(0.0, state["covenant_min_liquidity_usd"] - liq))
            state["st_investments_usd"] = float(state["st_investments_usd"] - sell)
            state["cash_usd"] = float(state["cash_usd"] + sell)
            action = f"SELL_ST_INVESTMENTS:{sell:.0f}"
        # recompute utilization target draw
        util_after = state["revolver_drawn_usd"] / state["revolver_limit_usd"]
        if util_after < state["max_revolver_utilization"]:
            headroom = state["max_revolver_utilization"] * state["revolver_limit_usd"] - state["revolver_drawn_usd"]
            draw = max(0.0, min(headroom, 2_500_000.0))
            if draw > 0:
                state["revolver_drawn_usd"] = float(state["revolver_drawn_usd"] + draw)
                state["cash_usd"] = float(state["cash_usd"] + draw)
                action = action + f" | DRAW_REVOLVER:{draw:.0f}"
    elif state["covenant_status"] == "WARNING":
        # If utilization already high, slow AP; else draw small buffer
        if util > (state["max_revolver_utilization"] - 0.03):
            slow = min(500_000.0, max(0.0, state["ap_due_7d_usd"] * 0.08))
            state["ap_due_7d_usd"] = float(max(0.0, state["ap_due_7d_usd"] - slow))
            action = f"SLOW_AP_PAYMENTS:{slow:.0f}"
        else:
            draw = 750_000.0
            # never exceed utilization cap
            cap_draw = state["max_revolver_utilization"] * state["revolver_limit_usd"] - state["revolver_drawn_usd"]
            draw = float(max(0.0, min(draw, cap_draw)))
            if draw > 0:
                state["revolver_drawn_usd"] = float(state["revolver_drawn_usd"] + draw)
                state["cash_usd"] = float(state["cash_usd"] + draw)
                action = f"DRAW_BUFFER:{draw:.0f}"
    else:
        action = "NO_ACTION"

    state["last_action"] = action
    return state

def node_notify(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    ev = state["last_event"]
    msg = {
        "run_id": state["run_id"],
        "event": ev,
        "cash_usd": round(state["cash_usd"], 2),
        "liquidity_usd": round(state["liquidity_usd"], 2),
        "revolver_utilization": round(state["revolver_utilization"], 4),
        "covenant_status": state["covenant_status"],
        "action": state["last_action"],
        "ts_utc": utc_now_iso(),
    }

    system = "You are a treasury monitoring assistant. Produce a terse operations note (3 bullets), no marketing."
    user = f"State snapshot JSON:\n{json.dumps(msg, sort_keys=True)}\n\nWrite the ops note."

    llm = anthropic_messages(
        api_key=ANTHROPIC_API_KEY,
        model=MODEL_NAME,
        system=system,
        user=user,
        max_tokens=160,
        temperature=0.2,
    )

    alert = {
        "ts_utc": utc_now_iso(),
        "severity": "HIGH" if state["covenant_status"] == "BREACH" else ("MED" if state["covenant_status"] == "WARNING" else "LOW"),
        "payload": msg,
        "llm_ok": bool(llm.get("ok", False)),
        "ops_note": llm.get("content_text", "")[:900],
    }
    state["alerts"] = list(state["alerts"]) + [alert]
    return state

COVENANT_GATE = AgentNode("COVENANT_GATE", node_covenant_gate)
ACTIONS = AgentNode("ACTIONS", node_liquidity_actions)
NOTIFY = AgentNode("NOTIFY", node_notify)

print("Covenant + actions + notify agents ready.")


Covenant + actions + notify agents ready.


##8.GRAPH

###8.1.OVERVIEW

**CELL 8 — LangGraph topology: the event-driven workflow, routing, bounded loops, and END**

Cell 8 is the architectural core of the notebook because it turns the individual nodes into a controlled system. Here we build the StateGraph, add nodes, define conditional routing, compile the graph, and render the diagram. For committee presentation, this cell is your strongest evidence: it shows the workflow is not a “bag of functions.” It is a defined process with explicit branching and termination.

The topology starts with `NEXT_EVENT` as the entry point. After pulling an event, we route based on stop conditions: if there is no event or we hit `max_steps`, we go to END. Otherwise we proceed. This is the first governance control: **the loop is bounded and justified**. In operational systems, you must have a stop condition; otherwise monitoring can become runaway automation.

The normal path is linear for clarity: APPLY_EVENT → METRICS → COVENANT_GATE. This ensures that after every event, metrics are recomputed before status is classified. Then `COVENANT_GATE` branches:
- If OK, we go to NOTIFY (routine reporting).
- If WARNING or BREACH, we go to ACTIONS.

The interesting part is what happens after ACTIONS. We loop back to METRICS and then to COVENANT_GATE again. This is a bounded stabilization pass: actions change the state, so we re-measure and re-classify. It is not an unbounded self-correction loop; it is a single structural pattern that mirrors professional behavior.

From NOTIFY, we go to ADVANCE, which increments the event index, then route either back to NEXT_EVENT or to END depending on whether we have more events and whether the step bound has been reached. This completes the event-driven cycle: pull event → process → decide → communicate → advance → repeat.

Two practical details matter. First, LangGraph has an internal recursion limit; in Cell 9 we configure it to avoid premature failure while still keeping the loop bounded by our own counters. Second, we render the Mermaid diagram immediately after compilation. This is not optional: visualization is required and the diagram must match topology. Because routing is explicit, the diagram is a faithful representation of what executes.

For teaching and governance, the key message is: Cell 8 makes the workflow explicit, inspectable, and defensible. Anyone can review the graph and understand the process. That is exactly what committees and risk functions want when you introduce automation into financial operations.


###8.2.CODE AND IMPLEMENTATION

In [18]:
# CELL 8/10 — LangGraph topology (event-driven workflow) with bounded loop + explicit END

def route_after_next_event(state: TreasuryState) -> str:
    # bounded loop + event-driven termination
    if state["steps"] >= state["max_steps"]:
        return "DONE"
    if state["last_event"] is None:
        return "DONE"
    return "HAVE_EVENT"

def route_after_metrics(state: TreasuryState) -> str:
    # Always evaluate covenant after metrics; "COVENANT_TEST" events simply force visibility
    return "CHECK"

def route_after_covenant(state: TreasuryState) -> str:
    if state["covenant_status"] == "BREACH":
        return "BREACH"
    if state["covenant_status"] == "WARNING":
        return "WARNING"
    return "OK"

def route_after_notify(state: TreasuryState) -> str:
    # advance event index and decide whether to process next event
    return "ADVANCE"

def node_advance(state: TreasuryState) -> TreasuryState:
    state = dict(state)
    if state["last_event"] is not None:
        state["event_idx"] += 1
    # refresh derived metrics after any actions (fast + deterministic)
    return state

def route_after_advance(state: TreasuryState) -> str:
    if state["steps"] >= state["max_steps"]:
        return "DONE"
    if state["event_idx"] >= len(state["events"]):
        return "DONE"
    return "CONTINUE"

ADVANCE = AgentNode("ADVANCE", node_advance)

g = StateGraph(TreasuryState)

g.add_node("NEXT_EVENT", NEXT_EVENT)
g.add_node("APPLY_EVENT", APPLY_EVENT)
g.add_node("METRICS", METRICS)
g.add_node("COVENANT_GATE", COVENANT_GATE)
g.add_node("ACTIONS", ACTIONS)
g.add_node("NOTIFY", NOTIFY)
g.add_node("ADVANCE", ADVANCE)

g.set_entry_point("NEXT_EVENT")

g.add_conditional_edges("NEXT_EVENT", route_after_next_event, {
    "HAVE_EVENT": "APPLY_EVENT",
    "DONE": END,
})

g.add_edge("APPLY_EVENT", "METRICS")

g.add_conditional_edges("METRICS", route_after_metrics, {
    "CHECK": "COVENANT_GATE",
})

g.add_conditional_edges("COVENANT_GATE", route_after_covenant, {
    "BREACH": "ACTIONS",
    "WARNING": "ACTIONS",
    "OK": "NOTIFY",
})

# If actions taken, recompute metrics → re-check covenant once (bounded correction loop).
# This is a single-pass stabilization step, not an unbounded “keep trying” loop.
g.add_edge("ACTIONS", "METRICS")

g.add_conditional_edges("NOTIFY", route_after_notify, {
    "ADVANCE": "ADVANCE",
})

g.add_conditional_edges("ADVANCE", route_after_advance, {
    "CONTINUE": "NEXT_EVENT",
    "DONE": END,
})

GRAPH = g.compile()

mermaid = _graph_to_mermaid(GRAPH)
display_langgraph_mermaid(mermaid, mermaid_version=str(CONFIG["mermaid_version"]))

print("Graph compiled. Nodes:", ["NEXT_EVENT","APPLY_EVENT","METRICS","COVENANT_GATE","ACTIONS","NOTIFY","ADVANCE","END"])


Graph compiled. Nodes: ['NEXT_EVENT', 'APPLY_EVENT', 'METRICS', 'COVENANT_GATE', 'ACTIONS', 'NOTIFY', 'ADVANCE', 'END']


##9.EXECUTION

###9.1.0VERVIEW

**CELL 9 — Execution: running the workflow safely and inspecting outputs**

Cell 9 runs the compiled graph on the initial state and prints a concise operational summary. In an agentic workflow, execution is not just “run a function.” It is “run a state machine until it reaches END.” This cell demonstrates that mode of operation and provides immediate feedback that the workflow behaved as intended.

A key improvement is the explicit `recursion_limit` configuration passed to `GRAPH.invoke`. LangGraph uses an internal recursion counter to prevent infinite loops. Even though our workflow is bounded by `max_steps` and `event_idx`, the internal limit can trigger earlier because each event can traverse multiple nodes, and warning/breach paths add extra transitions. Setting the recursion limit to a value derived from `max_steps` ensures the run completes while still remaining bounded by our own explicit controls. This is a professional stance: we do not remove safety; we align tool safety with workflow structure.

After execution, we print termination details: how many events were processed, how many steps were executed, and the final covenant status. This is a compact “run header” you can show in a committee meeting to prove the run ended correctly and did not thrash. We then print a final snapshot with key liquidity drivers: cash, short-term investments, revolver drawn, utilization, liquidity, AP/AR near-term values, and how many alerts were generated. This matches how treasury leaders think: they want the headline position and the key drivers, not an overwhelming log.

We also print the most recent alert and its ops note. This is where the AI contribution is visible: a short human-readable summary generated from state. Importantly, we also show whether the model call succeeded (`llm_ok`). If the key is missing, the notebook remains runnable and the note indicates LLM disabled. That keeps the demo robust.

From a pedagogy perspective, Cell 9 teaches that agentic workflows should produce **inspectable outputs** at multiple levels: headline metrics, recent alert, and later exported artifacts. In a production setting, you would push these outputs to dashboards, emails, or incident channels. Here, we print them for transparency.

For a committee, the critical story is: this cell proves the system is not speculative. It is runnable, bounded, and produces consistent operational outputs. It also shows the correct division of labor: deterministic monitoring and policy logic, with AI used for standardized reporting.


###9.2.CODE AND IMPLEMENTATION

In [20]:


# CELL 9/10 — Execute workflow + inspect key outputs (fast classroom run)

# A conservative upper bound:
# Each processed event can traverse:
# NEXT_EVENT -> APPLY_EVENT -> METRICS -> COVENANT_GATE
#   -> (OK) NOTIFY -> ADVANCE -> NEXT_EVENT  (≈ 6 transitions)
# or (WARNING/BREACH) ACTIONS -> METRICS -> COVENANT_GATE -> ... (adds ≈ 2-3)
# So ~9 per event worst-case. Multiply by max_steps, add headroom.
per_step_upper = 10
safe_recursion_limit = int(final_state["max_steps"] * per_step_upper + 50) if "final_state" in globals() else int(INIT_STATE["max_steps"] * per_step_upper + 50)

final_state: TreasuryState = GRAPH.invoke(
    INIT_STATE,
    config={"recursion_limit": safe_recursion_limit}
)

print("TERMINATION:",
      "event_idx", final_state["event_idx"], "/", len(final_state["events"]),
      "| steps", final_state["steps"], "/", final_state["max_steps"],
      "| covenant_status", final_state["covenant_status"],
      "| recursion_limit", safe_recursion_limit)

print("\nLAST SNAPSHOT:")
print(json.dumps({
    "cash_usd": round(final_state["cash_usd"], 2),
    "st_investments_usd": round(final_state["st_investments_usd"], 2),
    "revolver_drawn_usd": round(final_state["revolver_drawn_usd"], 2),
    "revolver_utilization": round(final_state["revolver_utilization"], 4),
    "liquidity_usd": round(final_state["liquidity_usd"], 2),
    "ap_due_7d_usd": round(final_state["ap_due_7d_usd"], 2),
    "ar_due_7d_usd": round(final_state["ar_due_7d_usd"], 2),
    "alerts_count": len(final_state["alerts"]),
}, indent=2, sort_keys=True))

print("\nMOST RECENT ALERT (if any):")
if final_state["alerts"]:
    a = final_state["alerts"][-1]
    print("severity:", a["severity"], "| llm_ok:", a["llm_ok"])
    print(a["ops_note"])
else:
    print("(none)")


TERMINATION: event_idx 18 / 18 | steps 18 / 30 | covenant_status OK | recursion_limit 350

LAST SNAPSHOT:
{
  "alerts_count": 18,
  "ap_due_7d_usd": 6500000.0,
  "ar_due_7d_usd": 4190848.99,
  "cash_usd": 14414519.0,
  "liquidity_usd": 14838926.99,
  "revolver_drawn_usd": 10500000.0,
  "revolver_utilization": 0.42,
  "st_investments_usd": 7733559.0
}

MOST RECENT ALERT (if any):
severity: LOW | llm_ok: True
**Treasury Operations Note**

• **Rate Move**: SOFR decreased 2 bps effective 2026-02-21 17:00 UTC; monitor impact on floating-rate obligations and cash positioning.

• **Liquidity Position**: $14.8M available liquidity with revolver at 42% utilization; covenant status OK.

• **Cash Balance**: $14.4M on hand; no immediate action required.


##10.AUDIT BUNDLE

###10.1.OVERVIEW

**CELL 10 — Artifact export: run_manifest.json, graph_spec.json, final_state.json**

Cell 10 closes the loop by exporting the artifacts that make this notebook “audit-grade” rather than “demo-grade.” In finance, a workflow run is not complete if you cannot reconstruct what happened. This cell ensures that every run produces evidence: what configuration was used, what topology executed, and what state resulted.

`run_manifest.json` captures run identity, timestamp, model lock, configuration hash, environment fingerprint (Python, platform, package versions), and run bounds (steps executed, events processed). This is your reproducibility anchor. If someone questions the result later, you can prove which exact settings and environment produced it. In an institutional setting, this is analogous to a model run log or a job control record.

`graph_spec.json` captures the workflow topology. We extract nodes and edges from Mermaid text into a structured representation. This is important because it decouples governance evidence from visualization rendering. Even if Mermaid fails on a future machine, the graph spec still states what nodes were connected and how. The spec also includes notes describing the architectural dimension and loop justification. Committees care about that because it explains why the system is safe and bounded.

`final_state.json` exports the full end state, including cash position, liquidity metrics, covenant status, alerts, and the trace. The trace is particularly valuable: it records node-by-node execution context. This means you can replay the narrative of the run: which event was processed, what actions were taken, and what status classifications occurred.

The cell writes files to the working directory and prints the filenames, making it obvious that artifacts were generated. In professional practice, you would likely also package these into a zip, store them in object storage, or attach them to a monitoring ticket. The notebook keeps it simple but demonstrates the discipline.

Compared to earlier notebooks, this artifact design is consistent: every notebook in the series emphasizes inspectable outputs. But Notebook 9’s artifacts are especially important because event-driven systems are continuous by nature. Continuous systems require stronger logging discipline: you must be able to answer “what happened between 10:00 and 16:00?” Exporting final state and trace is a minimal version of that.

In short, Cell 10 is what makes the notebook committee-ready. It produces durable evidence that the workflow ran under a specific configuration, followed a specific topology, and ended in a specific state—all of which can be independently reviewed.


###10.2.CODE AND IMPLEMENTATION

In [ ]:
# CELL 10/10 — Export required artifacts: run_manifest.json, graph_spec.json, final_state.json

def graph_spec_from_mermaid(mermaid_code: str) -> Dict[str, Any]:
    # Lightweight extraction: capture "A --> B" edges; preserve determinism.
    edges: List[Tuple[str, str]] = []
    nodes: Dict[str, Dict[str, Any]] = {}

    for line in mermaid_code.splitlines():
        s = line.strip()
        if "-->" in s:
            left, right = s.split("-->", 1)
            a = left.strip().split()[-1]
            b = right.strip().split()[0]
            a = a.replace("[", "").replace("]", "")
            b = b.replace("[", "").replace("]", "")
            edges.append((a, b))
            nodes.setdefault(a, {"id": a})
            nodes.setdefault(b, {"id": b})

    return {
        "format": "mermaid_flowchart",
        "nodes": sorted(nodes.keys()),
        "edges": [{"from": a, "to": b} for (a, b) in edges],
    }

run_manifest = {
    "run_id": RUN_ID,
    "ts_utc": utc_now_iso(),
    "model_lock": MODEL_NAME,
    "config": CONFIG,
    "config_hash": CONFIG_HASH,
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "versions": {k: _ver(k) for k in _REQUIRED},
        "api_key_present": bool(ANTHROPIC_API_KEY),
    },
    "bounds": {
        "max_steps": final_state["max_steps"],
        "steps_executed": final_state["steps"],
        "events_total": len(final_state["events"]),
        "events_processed": final_state["event_idx"],
    },
}

graph_spec = {
    "run_id": RUN_ID,
    "ts_utc": utc_now_iso(),
    "topology": graph_spec_from_mermaid(mermaid),
    "notes": {
        "architecture_dimension": "Event-driven workflow for treasury liquidity & covenant monitoring",
        "bounded_loop_justification": "Classroom safety + auditability: process at most max_steps events per run; prevent runaway monitoring loops.",
        "stabilization_pass": "After actions, metrics are recomputed and covenant gate re-evaluated once via METRICS->COVENANT_GATE path.",
    },
}

final_state_export = final_state  # already JSON-serializable TypedDict

with open("run_manifest.json", "w", encoding="utf-8") as f:
    json.dump(run_manifest, f, indent=2, sort_keys=True)

with open("graph_spec.json", "w", encoding="utf-8") as f:
    json.dump(graph_spec, f, indent=2, sort_keys=True)

with open("final_state.json", "w", encoding="utf-8") as f:
    json.dump(final_state_export, f, indent=2, sort_keys=True)

print("WROTE: run_manifest.json | graph_spec.json | final_state.json")
print("FILES:", sorted([p for p in os.listdir(".") if p.endswith(".json")]))


##11.CONCLUSION

**Conclusion — From Event-Driven Monitoring to Autonomous Treasury Operations (and How Notebook 9 Advances the Series)**

This notebook marks a practical turning point in the AA-FIN-LG-2026 progression because it shifts the architecture from “answering” and “drafting” workflows into **continuous operational monitoring**. Earlier notebooks established the fundamental building blocks of governed agentic systems: bounded retry loops (N1), hard suitability gates and early termination (N2), structured critique loops with evidence gaps (N3), tool-augmented reasoning around a hypothesis (N4), regime-aware control (N5), parallel committee aggregation (N6), hub-and-spoke production of investment banking deliverables (N7), and router + retrieval patterns for diligence Q&A (N8). Notebook 9 adds a new architectural dimension that finance leadership immediately recognizes: **time is not a parameter, it is the driver**. Treasury does not live in “one request, one response.” It lives in a stream of updates that must be processed consistently, quickly, and with escalation discipline.

The core achievement here is that we encoded a treasury monitoring process as a **state-driven event loop** in LangGraph, with explicit boundedness and artifact generation. The system behaves like a controlled operations workflow: it ingests the next event, applies it, recomputes metrics, evaluates covenant status, triggers policy actions when needed, drafts an operational note, and advances. The important point is that the topology is not decorative. It is the mechanism. The diagram itself becomes part of governance: it shows exactly what the system can do, what it will never do, and when it stops.

Compared to earlier notebooks, Notebook 9 has a different “center of gravity.” In N1–N3, the architecture’s main purpose was managing uncertainty in user input and evidence: missing information discovery, refusal boundaries, critique loops. In N4, it was about structuring tool calls around hypotheses and backtests. In N5, it was about policy behavior under regime changes. In N6 and N7, it was about multi-perspective production—committee votes and hub-and-spoke drafting. In N8, it was about routing queries to retrieval and response. Here, the new capability is that the system is no longer primarily “reactive to a user prompt.” It is **reactive to operational reality**, modeled as events. That is why this notebook reads less like a “chat workflow” and more like a **monitoring control system**.

At the same time, we intentionally kept the most sensitive part of treasury—executive decisions—outside the language model. The workflow’s routing and actions are rule-based and inspectable. The language model is limited to producing a short, consistent operations note from a structured snapshot. That design is appropriate for a governed educational series because it demonstrates the safe baseline: **AI communicates; state machines control**. But it also naturally invites the next question: how do we move from this governed baseline to architectures that are more autonomous?

Moving toward autonomy in event-driven systems means shifting three capabilities from deterministic logic into AI-mediated decisions, while keeping control and auditability intact:

1. **Autonomous interpretation of events.**  
   In this notebook, events arrive already classified (“CASH_IN,” “RATE_MOVE,” etc.). In a more autonomous system, the workflow would ingest raw messages—bank statements, ERP updates, email from lenders, market data ticks—and an AI node would normalize them into a structured event type with fields and confidence. This is where a model can add real operational value: converting messy operational inputs into clean treasury events. The governance requirement is that the system must store the extracted structure, confidence, and provenance, and must never hide uncertainty.

2. **Autonomous routing under policy constraints.**  
   Today, routing is hard-coded: metrics always flow into the covenant gate, and the covenant status determines whether we act or notify. A more autonomous architecture could allow an AI router node to decide which checks to run next, for example: “FX move detected, run exposure ladder check,” or “rate spike detected, reprice floating debt service forecast,” or “lender request received, compile covenant evidence pack.” This kind of routing autonomy becomes valuable as the organization’s treasury logic expands beyond a single covenant metric into a portfolio of checks. The control principle is that autonomy is not permissionless: the router must choose only among an approved set of nodes, and every chosen path must be logged and replayable.

3. **Autonomous action proposals and execution planning.**  
   In this notebook, actions are deterministic and simplistic: sell liquid investments, draw revolver within a cap, slow payables in warning. In a more autonomous system, the AI could generate multiple action options—each with rationale, expected liquidity impact, constraints, and risks—then route those options through approval gates. Autonomy does not have to mean the system executes trades or draws lines of credit by itself on day one. The professional path is staged autonomy:
   - **Stage A: AI proposes actions** (human approves).
   - **Stage B: AI executes low-risk actions** (pre-approved playbooks) and escalates high-risk actions.
   - **Stage C: AI manages a bounded action budget** under explicit risk limits and exception reporting.
   Each stage requires stronger controls: policy encoding, approval workflows, segregation of duties, and evidence packs.

This staged progression mirrors how we should think about autonomy across the whole notebook series. The series is not about “agents that do everything.” It is about **architectures that add capability while adding controls**. Notebook 9 is a strong foundation for autonomy precisely because it already has the correct skeleton: explicit state, explicit graph, bounded loops, and exported artifacts. Those features are what make autonomy feasible without losing governance.

In practical terms, the next steps toward autonomous event-driven treasury operations would look like architectural upgrades rather than prompt upgrades:

- **Add an “Event Normalizer” node** that uses the model to map raw inputs into structured TreasuryEvent objects, with confidence scores and redaction controls.
- **Add a “Policy Interpreter” node** that converts policy documents or covenant definitions into machine-checkable constraints, but only within a controlled schema.
- **Add a “Scenario Simulator” node** that runs what-if projections (next 7/14/30 days) triggered by certain event classes, then routes results to escalation.
- **Add a “Decision Proposal” node** that generates action alternatives, each tagged with risk category, approvals required, and expected impact.
- **Add explicit approval gates** as separate graph nodes: human review, CFO approval, lender notification. These become hard branching boundaries similar to the refusal/termination boundaries introduced in earlier notebooks.

This is also where the comparison to prior notebooks becomes useful in a committee setting. Notebook 2 taught the discipline of refusal boundaries: some questions must terminate early. Notebook 5 taught regime control: behavior changes under stress. Notebook 6 taught aggregation: multiple perspectives produce a final decision. These are exactly the patterns you would reuse when you introduce autonomy in treasury. An autonomous treasury agent must be able to refuse actions outside mandate (Notebook 2), adapt under stressed liquidity regimes (Notebook 5), and consult multiple “committee” perspectives before proposing actions (Notebook 6). Notebook 9 provides the operational substrate—events and time—and the earlier notebooks provide the governance patterns—gates, regimes, committees, critique loops, and retrieval.

So the real message of this notebook, and the reason it belongs in Chapter 9, is that it makes the architecture feel like a real financial system. It is no longer a static reasoning exercise. It is a controlled loop that mirrors the tempo of treasury life. The next step is not to “make the model smarter.” The next step is to **expand the graph into a supervised autonomy system**, where AI can classify events, propose routes, and draft action plans—while the organization retains control through explicit policy constraints, hard gates, bounded loops, and audit-grade artifacts.

If you want one committee-ready sentence to close with:

**Notebook 9 moves us from governed reasoning workflows to governed operations workflows: an event-driven, state-controlled treasury monitor. The path to autonomy is to let AI handle interpretation, routing, and action proposals inside a constrained graph, while preserving the same governance primitives we built across earlier notebooks—gates, bounded loops, committees, and audit artifacts.**
